In [ ]:
!pip install mlflow

In [107]:
import shutil
import os
import glob
import pandas as pd
import mlflow
#shutil.make_archive("mlruns", 'zip', "mlruns/")

import plotly.express as px
import matplotlib.pyplot as plt

In [112]:
out = []
out2 = {}
for p in glob.glob("mlruns/1/*/metrics/*"):
    
    run_id = p.split("/")[2]
    metric_type = p.split("/")[-1]
    
    df = pd.read_csv(p, sep = " ", names=["timestep", "value", "step"])
    df["run_id"] = run_id
    df["metric_type"] = metric_type
    
    out.append(df)
    
    out2[run_id] = mlflow.get_run(run_id)._data.params

out = pd.concat(out, axis=0).set_index(["step","run_id", "metric_type"])
out = out["value"].unstack()

out2 = pd.DataFrame(out2).T
out = out.reset_index().join(out2, on = "run_id")

In [113]:
#Filtering based on the best accuracy_test for each run.
out["test_score"] = out.sort_values("accuracy_test", ascending = False).groupby("run_id").cumcount()
out = out.query("test_score == 0")

In [114]:
a = px.scatter(out.query("test_score==0"), x = "accuracy_test", y = "mae_test", hover_data = ["lr", "cb_matrix"])

with open("ordinal_performances.html","w") as f:
    f.write(a.to_html())